In [24]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt
from sklearn import preprocessing
import os  # there is an issue with running some machine learning application on MacOX and this is to resolve it
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import warnings
warnings.filterwarnings("ignore")

In [2]:
# metadata
subset_rate, test_portion = 0.3, 0.2

In [3]:
dataset = pd.read_csv('../datasets/train.csv', low_memory = False)
dataset['IsRusher'] = False
dataset.loc[dataset.NflId == dataset.NflIdRusher, 'IsRusher'] = True
dataset['OffenseDefense'] = 'D'
dataset['OffenseDefense'] = np.where((dataset.PossessionTeam == dataset.HomeTeamAbbr) & \
                                       (dataset.Team == 'home'), 'O', 
                                np.where((dataset.PossessionTeam == dataset.VisitorTeamAbbr) & \
                                        (dataset.Team == 'away'), 'O', 'D'))
# with %%time using pandas is taking longer time
# dataset['OffenseDefense'] = dataset[((dataset.PossessionTeam == dataset.HomeTeamAbbr) & (dataset.Team == 'home')) | \
#                                     ((dataset.PossessionTeam == dataset.VisitorTeamAbbr) & (dataset.Team == 'away'))] ='O'
game_id = random.sample(list(dataset.GameId.unique()), int(subset_rate * len(dataset.GameId.unique())))
test_id, train_id = game_id[:int(test_portion * len(game_id))], game_id[int(test_portion * len(game_id)):]
train_plays = dataset.loc[dataset.GameId.isin(train_id), 'PlayId'].unique()
test_plays = dataset.loc[dataset.GameId.isin(test_id), 'PlayId'].unique()

In [4]:
def transform_train_test(dataset, play_id_lists):
    X = []
    Y = []
    for play_id in play_id_lists:
        game_data = dataset[dataset.PlayId == play_id]
        cols_to_dl = ['PlayId', 'X', 'Y', 'S', 'A', 'Orientation', 'IsRusher', 'OffenseDefense', 'PlayDirection', 'Yards']
        spatial_data = game_data[cols_to_dl]
        rusher_data = spatial_data[spatial_data.IsRusher]
        if spatial_data.PlayDirection.values[0] == 'right':
            spatial_data.loc[:, ['X', 'Y']] = spatial_data.loc[:,['X', 'Y']].values - rusher_data[['X', 'Y']].values
        else:
            spatial_data.loc[:,['X', 'Y']] = rusher_data.loc[:,['X', 'Y']].values - spatial_data.loc[:,['X', 'Y']].values
        spatial_data['RusherDistance'] = np.sqrt(np.square(spatial_data.X) + np.square(spatial_data.Y))
        spatial_data = spatial_data.sort_values(by = ['OffenseDefense', 'RusherDistance'])
        dl_input = list(pd.concat([spatial_data.X, spatial_data.Y, spatial_data.S, spatial_data.A, 
                                  spatial_data.Orientation, spatial_data.RusherDistance]))
        X.append(dl_input)
        dl_output = []
        dl_output[:199] = [0] * 199
        dl_output[99 + int(spatial_data.Yards.values[0])] = 1
        Y.append(dl_output)
    return torch.FloatTensor(X), torch.FloatTensor(Y), scaler


In [5]:
x_train, y_train, scaler = transform_train_test(train_plays)
x_test, y_test, _ = transform_train_test(test_plays, is_train = False, scaler = scaler)

In [6]:
n_in, n_h, n_out, batch_size, lr, epoch_size = x_train.shape[1], 200, y_train.shape[1], 32, 0.01, 100
model = nn.Sequential(nn.Linear(n_in, n_h),
                      nn.Sigmoid(), 
                      nn.Linear(n_h, n_out), 
                      nn.Softmax()
                     )
criterion = nn.MSELoss()
optim = torch.optim.SGD(model.parameters(), lr = lr)

In [8]:
for epoch in range(epoch_size):
    y_pred = model(x_train)
    
    # get the loss function
    loss = criterion(y_pred, y_train)
    if epoch % 100 == 0:
        print("epoch %s loss value: %.3f"%(epoch, loss.item()))
        
    # zero the gradient
    optim.zero_grad()
    
    # back propogate
    loss.backward()
    
    # update the parameter
    optim.step()
    
test_pred = model(x_test)
test_MSE = torch.mean((test_pred - y_test)**2)
print('test mean squared error is %.3f' %(test_MSE.item()))

epoch 0 loss value: nan
test mean squared error is nan


In [11]:
y_pred = model(x_train)

In [23]:
x_train[0]

tensor([ 5.7400e+00,  5.2300e+00,  6.7200e+00,  5.0700e+00,  8.5400e+00,
         9.5900e+00,  1.2270e+01,  1.4450e+01,  1.3440e+01,  7.9500e+00,
         2.3560e+01,  0.0000e+00,  3.9600e+00,  1.1600e+00,  4.8200e+00,
         5.7400e+00,  5.1000e+00,  6.2500e+00,  7.9000e+00,  7.8200e+00,
         8.9200e+00,  6.5600e+00,  9.8000e-01,  3.0400e+00, -1.9300e+00,
         6.9900e+00, -2.9800e+00,  1.9300e+00, -5.9900e+00,  8.6600e+00,
        -1.5120e+01,  1.8800e+01, -1.3400e+00,  0.0000e+00, -1.9800e+00,
         4.6100e+00,  2.1000e-01, -2.0000e+00,  3.9400e+00,  2.8400e+00,
         1.3700e+00, -4.4400e+00, -1.5850e+01,  1.8170e+01,  3.6300e+00,
         4.8700e+00,  2.7300e+00,  4.9200e+00,  3.4400e+00,  4.1300e+00,
         2.1000e+00,  3.3100e+00,  1.1600e+00,  3.9000e+00,  2.4100e+00,
         6.3200e+00,  5.6600e+00,  3.2700e+00,  4.0400e+00,  2.3300e+00,
         5.4200e+00,  4.4400e+00,  5.0700e+00,  1.2300e+00,  4.3400e+00,
         4.6900e+00,  2.7200e+00,  2.1600e+00,  1.6